Câu 1 

In [2]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng ví dụ
cursor.execute('''
CREATE TABLE my_table (
    A REAL,
    B REAL
)
''')

# Chèn dữ liệu mẫu
data = [
    (1, 2),
    (2, 4),
    (3, 6),
    (4, 8),
    (5, 10)
]
cursor.executemany('INSERT INTO my_table (A, B) VALUES (?, ?)', data)
conn.commit()

query = '''
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) * 
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS pearson_r
FROM my_table
'''

result = cursor.execute(query).fetchone()
print("Hệ số tương quan Pearson r_AB:", result[0])



Hệ số tương quan Pearson r_AB: 0.9999999999999999


Câu 2 

In [3]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu dạng quan hệ
cursor.execute("""
CREATE TABLE car_scores (
    Day TEXT,
    CarModel TEXT,
    Score REAL
)
""")

# Dữ liệu
data = [
    ('Day 1', 'A', 8), ('Day 1', 'B', 9), ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6), ('Day 3', 'B', 7), ('Day 3', 'C', 8),
    ('Day 4', 'A', 7), ('Day 4', 'B', 6), ('Day 4', 'C', 5),
]
cursor.executemany("INSERT INTO car_scores (Day, CarModel, Score) VALUES (?, ?, ?)", data)
conn.commit()


In [4]:
# Tạo bảng tạm có phân nhóm điểm (Low / Medium / High)
query_grouped = """
SELECT
    *,
    CASE
        WHEN Score <= 6.5 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS ScoreGroup
FROM car_scores
"""

df_grouped = pd.read_sql_query(query_grouped, conn)
print(df_grouped)


      Day CarModel  Score ScoreGroup
0   Day 1        A    8.0     Medium
1   Day 1        B    9.0       High
2   Day 1        C    7.0     Medium
3   Day 2        A    7.5     Medium
4   Day 2        B    8.5       High
5   Day 2        C    7.0     Medium
6   Day 3        A    6.0        Low
7   Day 3        B    7.0     Medium
8   Day 3        C    8.0     Medium
9   Day 4        A    7.0     Medium
10  Day 4        B    6.0        Low
11  Day 4        C    5.0        Low


In [5]:
# Bảng chéo giữa CarModel và ScoreGroup
ct_model = pd.crosstab(df_grouped['CarModel'], df_grouped['ScoreGroup'])
print("Bảng tần số theo mẫu xe:\n", ct_model)

# Kiểm định chi bình phương
chi2_model, p_model, dof_model, _ = chi2_contingency(ct_model)
print("\n[Kết quả kiểm định theo mẫu xe]")
print(f"Chi-squared = {chi2_model:.4f}, p-value = {p_model:.4f}")


Bảng tần số theo mẫu xe:
 ScoreGroup  High  Low  Medium
CarModel                     
A              0    1       3
B              2    1       1
C              0    1       3

[Kết quả kiểm định theo mẫu xe]
Chi-squared = 5.1429, p-value = 0.2730


In [6]:
# Bảng chéo giữa Day và ScoreGroup
ct_day = pd.crosstab(df_grouped['Day'], df_grouped['ScoreGroup'])
print("\nBảng tần số theo ngày:\n", ct_day)

# Kiểm định chi bình phương
chi2_day, p_day, dof_day, _ = chi2_contingency(ct_day)
print("\n[Kết quả kiểm định theo ngày]")
print(f"Chi-squared = {chi2_day:.4f}, p-value = {p_day:.4f}")



Bảng tần số theo ngày:
 ScoreGroup  High  Low  Medium
Day                          
Day 1          1    0       2
Day 2          1    0       2
Day 3          0    1       2
Day 4          0    2       1

[Kết quả kiểm định theo ngày]
Chi-squared = 6.0952, p-value = 0.4126


Câu 3 

In [7]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
""")

# Thêm dữ liệu mẫu
data = [(1, 830), (2, 1445), (3, 5), (4, 0), (5, 123)]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", data)
conn.commit()


In [8]:
query = """
SELECT 
    id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS departure_time_formatted
FROM flights
"""
df = pd.read_sql_query(query, conn)
print(df)


   id  departure_time departure_time_formatted
0   1             830                    08:30
1   2            1445                    14:45
2   3               5                    00:05
3   4               0                    00:00
4   5             123                    01:23


Câu 4 

In [9]:
import sqlite3
import pandas as pd
import numpy as np

# Tạo kết nối SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng có cột value
cursor.execute("""
CREATE TABLE data (
    id INTEGER PRIMARY KEY,
    value REAL
)
""")

# Dữ liệu có vài ngoại lệ
data = [(1, 10), (2, 12), (3, 11), (4, 10), (5, 300), (6, 9), (7, 10), (8, 11), (9, 305)]
cursor.executemany("INSERT INTO data (id, value) VALUES (?, ?)", data)
conn.commit()


In [10]:
# Lấy dữ liệu
df = pd.read_sql_query("SELECT * FROM data", conn)

# Tính median
median = df['value'].median()

# Tính MAD
mad = (df['value'] - median).abs().median()

# Ngưỡng: 1.5 * MAD
threshold = 1.5 * mad
print(f"Median = {median}, MAD = {mad}, Threshold = {threshold}")


Median = 11.0, MAD = 1.0, Threshold = 1.5


In [11]:
query = f"""
SELECT *,
       ABS(value - {median}) AS deviation_from_median
FROM data
WHERE ABS(value - {median}) > {threshold}
"""

outliers = pd.read_sql_query(query, conn)
print(outliers)


   id  value  deviation_from_median
0   5  300.0                  289.0
1   6    9.0                    2.0
2   9  305.0                  294.0


Câu 5 

In [ ]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng bệnh nhân
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
)
""")

# Thêm dữ liệu mẫu
data = [
    (1, 'Ngo ', 80 , 170),
    (2, 'Ngo', 80.2, 170),
    (3, 'Tran', 55, 165),
    (4, 'Ngo', 65, 170),
    (5, 'Le', 60, 170)
]
cursor.executemany("INSERT INTO Patient (id, last_name, weight, height) VALUES (?, ?, ?, ?)", data)
conn.commit()


In [14]:
query = """
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    p1.last_name,
    p1.weight AS weight1,
    p2.weight AS weight2,
    ABS(p1.weight - p2.weight) AS weight_diff,
    CASE
        WHEN p1.last_name = p2.last_name AND ABS(p1.weight - p2.weight) <= 0.5 THEN 1
        ELSE 0
    END AS is_same_person
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
"""
df = pd.read_sql_query(query, conn)
print(df)


   id1  id2 last_name  weight1  weight2  weight_diff  is_same_person
0    1    2      Ngo      80.0     80.2          0.2               0
1    1    3      Ngo      80.0     55.0         25.0               0
2    1    4      Ngo      80.0     65.0         15.0               0
3    1    5      Ngo      80.0     60.0         20.0               0
4    2    3       Ngo     80.2     55.0         25.2               0
5    2    4       Ngo     80.2     65.0         15.2               0
6    2    5       Ngo     80.2     60.0         20.2               0
7    3    4      Tran     55.0     65.0         10.0               0
8    3    5      Tran     55.0     60.0          5.0               0
9    4    5    Nguyen     65.0     60.0          5.0               0
